In [ ]:
##Code to plot performance trends over Minority Training Ratios and trials

In [ ]:
import os, os.path
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
#Load saved stats
root_checkpoint_dir = './all_checkpoints/' #This directory contains subdirectories, one for each trial (each random seed). The directory for each trial contains subdirectories for each minority training fraction.
runs = os.listdir(root_checkpoint_dir)

label = 'dark' #***Naming convention: DS-1: dark, DS-2: dark, DS-3: male, DS-4: Female, DS-5:male, DS-6: chicken
ll = len(label)
samps = 11 #no. of minority training ratios used

name_save_majority = 'test_accuracy_L.npy' #***Majority group stats; naming convention: DS-1: test_accuracy_L.npy, DS-2: test_accuracy_L.npy, DS-3: test_accuracy_M.npy, DS-4: test_accuracy_M.npy, DS-5: test_accuracy_F.npy, DS-6: test_accuracy_C.npy
name_save_minority = 'test_accuracy_D.npy' #***Minority group stats; naming convention: DS-1: test_accuracy_D.npy, DS-2: test_accuracy_D.npy, DS-3: test_accuracy_F.npy, DS-4: test_accuracy_F.npy, DS-5: test_accuracy_M.npy, DS-6: test_accuracy_H.npy

plot_maj_test = np.zeros([samps,2,len(runs)])

zz=0
for run in runs:
    conf = root_checkpoint_dir+run+'/'
    
    fs = os.listdir(conf)
    #get indices to reorder
    fs_new = []
    l_new = []
    for f in fs:
      if os.path.isdir(conf+f):
        f_t = float(f[ll:])
        l_new.append(f_t)
        fs_new.append(f)

    fs = fs_new

    l_new = np.array(l_new)
    ixs = np.flip(np.argsort(l_new))
    l_new = np.flip(np.sort(l_new))
    print(l_new)

    fs =[fs[ix] for ix in ixs] #fs[ixs]
    #flip to get in order of minority fraction
    #fs.reverse()
#     print(fs)


    i = 0
    for f in fs:
      if os.path.isdir(conf+f):
#         print(f)
        path = conf+f

        v2 = np.load(path+'/'+name_save_majority) 
        v3 = np.load(path+'/'+name_save_minority) 

        t_ix_max = np.argmax(v2)

        plot_maj_test[i,0,zz] = v2[t_ix_max]
        plot_maj_test[i,1,zz] = v3[t_ix_max]
        i+=1
    zz=zz+1

In [ ]:
l_dash = l_new #***For DS-1, DS-2, DS-4, DS-5 (saved wrt minority fraction)
# l_dash = 1-l_new #***For DS-3, DS-6 (saved wrt majority fraction)

In [ ]:
#plot plots- for 
plt.style.use('seaborn-whitegrid')

plot_vals = plot_maj_test

avg_acc = np.mean(plot_vals[:,0,:],axis=-1) #Average accuracy

std_maj = np.sqrt(np.var(plot_vals[:,0,:]-np.reshape(np.mean(plot_vals[:,0,:],axis=0),(-1,plot_vals.shape[-1])),axis=-1))


errs = std_maj #error bounds

plt.figure(figsize=(8,8), dpi=200)

plt.plot(l_dash,avg_acc,'k-o',linewidth=3)
plt.plot(l_dash,avg_acc[-1]*np.ones(l_dash.shape),'r--')

plt.fill_between(l_dash, avg_acc - errs, avg_acc + errs,
                 color='gray', alpha=0.2)

plt.yticks(fontsize=25,fontname='Serif')
plt.xticks(fontsize=25,fontname='Serif')
plt.grid(False)
plt.show()

